# Data processing - Qredo transactions

#### Maria Silva, May 2023

In [1]:
import os
import numpy as np
import pandas as pd

## QRDO transactions

### Read raw data

In [2]:
doc1_file = os.path.realpath("../data/transaction_export.csv")

raw_qrdo_df = pd.read_csv(doc1_file)
raw_qrdo_df["created_at"] = pd.to_datetime(raw_qrdo_df["timestamp"], unit="s", utc=True)
raw_qrdo_df["day"] = raw_qrdo_df["created_at"].dt.date

raw_qrdo_df.head(1).T

,0
tx_id,2KPa6iXYUZ4saCFAty3syYW2iOv
tx_type,transferIn
chain_tx_id,A29FCEDAF1A26EAE5BB70AC8583589CE074B8262DE9E07...
wallet_id,2aa37GdQutBTkgsYTRthkP55rxNRtAJnP2Fe9pu2BMbq
fund_id,1xKBXuUyuCHAeeyX5LAhqo2gHQE
asset,QRDO
asset_scale,100000000
amount,161222790
fees,0
net_amount,161222790


In [3]:
len(raw_qrdo_df[raw_qrdo_df["amount"] <0])

8

In [4]:
raw_qrdo_df.groupby("benefit_of")["tx_id"].nunique().sort_values(ascending=False)

benefit_of
Staking event           355789
Vesting event           154754
QredoStation              1168
PowerSwap                  270
1                          170
                         ...  
Ledger                       1
MGL UK                       1
MKG Enterprises Corp         1
MOHAN                        1
you                          1
Name: tx_id, Length: 309, dtype: int64

### Clean raw data

Some notes here:

* Some transferIn + transferOut  relate to staking and vesting events, which are filtered using the "benefit_of" field. We will give these transactions their own transaction type
* The other transferIn + transferOut are layer 2 transactions. For each transferIn, there is an equivalent transferOut. We will consider the transferOut only and rename these transactions as QL2.
* swapIn and swapOut will be excluded (there is only 4 of these transactions)

In [5]:
filter_df = raw_qrdo_df[(raw_qrdo_df["asset"]=="QRDO") & (raw_qrdo_df["amount"]>0)].copy()

staking_df = filter_df[(filter_df["benefit_of"] == "Staking event") & (filter_df["tx_type"] == "transferIn")].copy()
staking_df["tx_type"] = "staking"

# Need to double check this!!!
vesting_df = filter_df[filter_df["benefit_of"] == "Vesting event"].copy()
vesting_df["tx_type"] = np.where(vesting_df["tx_type"]=="transferIn", "vestingIn", "vestingOut")

ql2_df = filter_df[(~filter_df["benefit_of"].isin(["Staking event", "Vesting event"])) & (filter_df["tx_type"] == "transferOut")].copy()
ql2_df["tx_type"] = "ql2"

others_df = filter_df[filter_df["tx_type"].isin(["deposit", "withdraw"])].copy()

In [6]:
qrdo_df = pd.concat([staking_df, ql2_df, vesting_df, others_df], ignore_index=True)

qrdo_df["tx_group"] = "qrdo"
qrdo_df["token"] = "QRDO"

# Need to fix this!!!
qrdo_df["amount_usd"] = qrdo_df["amount"]/qrdo_df["asset_scale"]

qrdo_df = qrdo_df.sort_values("timestamp").reset_index(drop=True)
qrdo_df = qrdo_df[["created_at", "day","tx_group", "tx_type", "fund_id", "wallet_id", "token", "amount_usd"]].copy()

qrdo_df.head()

,created_at,day,tx_group,tx_type,fund_id,wallet_id,token,amount_usd
0,2021-08-26 11:28:23+00:00,2021-08-26,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,100.0
1,2021-08-26 11:55:58+00:00,2021-08-26,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,999900.0
2,2021-08-30 08:48:37+00:00,2021-08-30,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,100.0
3,2021-08-30 08:54:12+00:00,2021-08-30,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,100.0
4,2021-08-30 10:56:44+00:00,2021-08-30,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,4000.0


### Compute aggregation

In [7]:
tx_count_df = qrdo_df.groupby(["day","tx_group", "tx_type"]).size()
tx_vol_df = qrdo_df.groupby(["day","tx_group", "tx_type"])["amount_usd"].sum()

agg_qrdo_df = pd.DataFrame()
agg_qrdo_df["tx_count"] = tx_count_df
agg_qrdo_df["amount_usd"] = tx_vol_df
agg_qrdo_df = agg_qrdo_df.reset_index()

agg_qrdo_df.head()

,day,tx_group,tx_type,tx_count,amount_usd
0,2021-08-26,qrdo,withdraw,2,1.000000e+06
1,2021-08-30,qrdo,withdraw,6,2.500410e+07
2,2021-09-05,qrdo,withdraw,2,2.002000e+05
3,2021-09-06,qrdo,deposit,15,2.114224e+04
4,2021-09-06,qrdo,withdraw,1,3.754100e+04


## MMI transactions

### Read raw data

In [8]:
doc2_file = os.path.realpath("../data/Web3_Transactions.xlsx")

raw_mmi_df = pd.read_excel(doc2_file, sheet_name="external_tx_succes_parse") 
raw_mmi_df["created_at"] = pd.to_datetime(raw_mmi_df["created_at"])
raw_mmi_df.head(1).T

,0
wallet_id,FoY7Z2wrGdQ6gkQPhDE5p27CpcWsuDopHQPPi95cx9BL
wallet_address,0x587e728Ec71e020FCbd14fAE8f813Bd4d0D97694
account_id,2AHTbZhl83mYZpZu0pzgajzA4mL
fund_id,2Lzi48TtxpDMFyW4QQGaJH8eiaI
chain_transactions,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
token,USDC
tx_hash,0x586307d94bc0b699c29a355a9f52141bc44c01c47807...
tx_type,send
amount,1500049.65522
token_price,0.9999


### Clean raw data

In [9]:
mmi_df = raw_mmi_df.copy()

mmi_df["amount_usd"] = mmi_df["transactions_usd"]
mmi_df["day"] = mmi_df["created_at"].dt.date
mmi_df["tx_group"] = "mmi"
mmi_df["tx_type"] = "mmi"

mmi_df = mmi_df.sort_values("created_at").reset_index(drop=True)
mmi_df = mmi_df[["created_at", "day","tx_group", "tx_type", "fund_id", "wallet_id", "token", "amount_usd"]].copy()

mmi_df.head()

,created_at,day,tx_group,tx_type,fund_id,wallet_id,token,amount_usd
0,2021-10-28 16:36:05+00:00,2021-10-28,mmi,mmi,1zPlNijXj9zVbw7gPtWyk0vFmlp,4kxr2miAfWtdWNPDaiqUfNyfLvpKBkd9jTjdXv8QJDvb,ETH,380.00200
1,2021-10-28 16:41:17+00:00,2021-10-28,mmi,mmi,1zPlNijXj9zVbw7gPtWyk0vFmlp,4kxr2miAfWtdWNPDaiqUfNyfLvpKBkd9jTjdXv8QJDvb,ETH,378.94200
2,2021-10-28 16:43:44+00:00,2021-10-28,mmi,mmi,1zPlNijXj9zVbw7gPtWyk0vFmlp,4kxr2miAfWtdWNPDaiqUfNyfLvpKBkd9jTjdXv8QJDvb,ETH,0.00000
3,2021-10-28 16:54:46+00:00,2021-10-28,mmi,mmi,1zPlNijXj9zVbw7gPtWyk0vFmlp,4kxr2miAfWtdWNPDaiqUfNyfLvpKBkd9jTjdXv8QJDvb,ETH,190.00100
4,2021-10-30 18:25:24+00:00,2021-10-30,mmi,mmi,1zzbtPYYlNWu0RIkNjjZ0kCGLIy,CH7BHpvF89YdKwSNn8wK4UxMNLwQQCDqYMtu97gXpuLp,ETH,1.59351


### Compute aggregation

In [10]:
tx_count_df = mmi_df.groupby(["day","tx_group", "tx_type"]).size()
tx_vol_df = mmi_df.groupby(["day","tx_group", "tx_type"])["amount_usd"].sum()

agg_mmi_df = pd.DataFrame()
agg_mmi_df["tx_count"] = tx_count_df
agg_mmi_df["amount_usd"] = tx_vol_df
agg_mmi_df = agg_mmi_df.reset_index()

agg_mmi_df.head()

,day,tx_group,tx_type,tx_count,amount_usd
0,2021-10-28,mmi,mmi,4,9.489450e+02
1,2021-10-30,mmi,mmi,2,1.752861e+00
2,2021-11-16,mmi,mmi,10,7.970237e+07
3,2021-11-18,mmi,mmi,11,2.315842e+71
4,2021-11-20,mmi,mmi,1,5.148976e+00


## Join all data

In [12]:
tx_df = pd.concat([qrdo_df, mmi_df], ignore_index=True)
tx_df = tx_df.sort_values("created_at").reset_index(drop=True)

file = os.path.realpath("../data/clean_tx.csv")
tx_df.to_csv(file, index=False)

tx_df.head()

,created_at,day,tx_group,tx_type,fund_id,wallet_id,token,amount_usd
0,2021-08-26 11:28:23+00:00,2021-08-26,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,100.0
1,2021-08-26 11:55:58+00:00,2021-08-26,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,999900.0
2,2021-08-30 08:48:37+00:00,2021-08-30,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,100.0
3,2021-08-30 08:54:12+00:00,2021-08-30,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,100.0
4,2021-08-30 10:56:44+00:00,2021-08-30,qrdo,withdraw,1uw8EJTYRzHgTptItHhwq0juyeZ,2RPVyYFg2kuw7Ct4gL2yGjtV8sEkkKyoJ3r6LVqr8AMx,QRDO,4000.0


In [14]:
agg_df = pd.concat([agg_qrdo_df, agg_mmi_df], ignore_index=True)
agg_df = agg_df.sort_values("day").reset_index(drop=True)

file = os.path.realpath("../data/agg_tx.csv")
agg_df.to_csv(file, index=False)

agg_df.head()

,day,tx_group,tx_type,tx_count,amount_usd
0,2021-08-26,qrdo,withdraw,2,1.000000e+06
1,2021-08-30,qrdo,withdraw,6,2.500410e+07
2,2021-09-05,qrdo,withdraw,2,2.002000e+05
3,2021-09-06,qrdo,deposit,15,2.114224e+04
4,2021-09-06,qrdo,withdraw,1,3.754100e+04


In [18]:
agg_df.groupby("tx_type")["tx_count"].sum().sort_values(ascending=False)

tx_type
staking       352105
vestingOut    154754
vestingIn     153212
mmi            85633
withdraw       29722
ql2            27216
deposit         9874
Name: tx_count, dtype: int64